<a href="https://colab.research.google.com/github/Masum06/Text2App/blob/master/text2app_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Masum06/Text2App.git

Cloning into 'Text2App'...
remote: Enumerating objects: 339, done.
remote: Total 339 (delta 0), reused 0 (delta 0), pack-reused 339
Receiving objects: 100% (339/339), 150.43 MiB | 50.24 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
cd Text2App

[Errno 2] No such file or directory: 'Text2App'
/content/Text2App


In [8]:
import pandas as pd
import os

In [9]:
# For Colab

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Synthesized_Data.zip to Synthesized_Data (1).zip
User uploaded file "Synthesized_Data.zip" with length 1647135 bytes


In [7]:
ls -l

total 2060
drwxr-xr-x 4 root root    4096 Mar 25 08:53 Media/
drwxr-xr-x 2 root root    4096 Mar 25 08:53 model_checkpoints/
drwxr-xr-x 7 root root    4096 Mar 25 08:53 OpenNMT-py-legacy/
-rw-r--r-- 1 root root      86 Mar 25 08:53 README.md
-rw-r--r-- 1 root root      62 Mar 25 08:53 requirements.txt
-rw-r--r-- 1 root root   39691 Mar 25 08:53 sar_parser.py
drwxr-xr-x 2 root root    4096 Mar 25 08:48 Synthesized_Data/
-rw-r--r-- 1 root root     184 Mar 25 08:56 Synthesized_Data.zip
-rw-r--r-- 1 root root 1286244 Mar 25 08:53 text2app_data_synthesis.ipynb
-rw-r--r-- 1 root root    2281 Mar 25 08:53 Text2App.py
-rw-r--r-- 1 root root  744673 Mar 25 08:53 text2app_translate.ipynb


In [10]:
!unzip "Synthesized_Data (1).zip"

Archive:  Synthesized_Data (1).zip
  inflating: Synthesized_Data/nl_sar_test.csv  
  inflating: Synthesized_Data/nl_sar_train.csv  
  inflating: Synthesized_Data/nl_sar_valid.csv  


In [ ]:
if not os.path.exists("./Synthesized_Data"):
  os.makedirs("./Synthesized_Data")

* Before training, data must be synthesized with the data synthesizing notebook

* Please ensure that your train, valid and test data files are present in the Synthesized_Data directory in csv format.

* Make sure they are named "nl_sar_train.csv", "nl_sar_valid.csv" and "nl_sar_test.csv" respectively


In [11]:
syn_data_dir = os.listdir("./Synthesized_Data") 
if "nl_sar_train.csv" in syn_data_dir and "nl_sar_valid.csv" in syn_data_dir and "nl_sar_test.csv" in syn_data_dir:
  train = pd.read_csv('./Synthesized_Data/nl_sar_train.csv', error_bad_lines=False)
  valid = pd.read_csv('./Synthesized_Data/nl_sar_valid.csv', error_bad_lines=False)
  test = pd.read_csv('./Synthesized_Data/nl_sar_test.csv', error_bad_lines=False)
  print("Data found!")
else:
  print("Data not found!\nPlease ensure that your train, valid and test data files are present in the Synthesized_Data directory in csv format.")

Data found!


In [12]:
data = {
    "train": train,
    "test": test,
    "valid": valid
}

In [13]:
!mkdir -p data/

In [15]:
!rm data/*.txt

rm: cannot remove 'data/*.txt': No such file or directory


In [16]:
for key in data.keys():
  fp_input = open("data/src_{}.txt".format(key), 'w+')
  fp_input.write('\n'.join(list(data[key]['NL'].values)))
  fp_input.close

  fp_input = open("data/tgt_{}.txt".format(key), 'w+')
  fp_input.write('\n'.join(list(data[key]['SAR'].values)))
  fp_input.close

In [17]:
!pip install OpenNMT-py

     |████████████████████████████████| 215kB 18.3MB/s 
     |████████████████████████████████| 2.6MB 30.5MB/s 
     |████████████████████████████████| 276kB 53.5MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 748.8MB 25kB/s 
     |████████████████████████████████| 1.2MB 43.5MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=cfedbcc02392557e8a77551afaa198db7f6aaeadd004d11a4943fba5d45d420d
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=7c35549bc31393631146c253d3f281ec26b7d4417d00e981204eb125924d473f
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2

In [18]:
!export CUDA_VISIBLE_DEVICES

### Train with Copy Mechanism

In [19]:
!mkdir -p ./text2app_models

In [20]:
!python OpenNMT-py-legacy/preprocess.py -train_src data/src_train.txt -train_tgt data/tgt_train.txt \
-valid_src data/src_valid.txt -valid_tgt data/tgt_valid.txt -src_seq_length 85 -tgt_seq_length 50 -save_data data/preprocessed_data

# -dynamic_dict 

[2021-03-25 09:44:11,957 INFO] Extracting features...
[2021-03-25 09:44:11,960 INFO]  * number of source features: 0.
[2021-03-25 09:44:11,960 INFO]  * number of target features: 0.
[2021-03-25 09:44:11,960 INFO] Building `Fields` object...
[2021-03-25 09:44:11,960 INFO] Building & saving training data...
[2021-03-25 09:44:12,350 INFO] Building shard 0.
[2021-03-25 09:44:13,755 INFO]  * saving 0th train data shard to data/preprocessed_data.train.0.pt.
[2021-03-25 09:44:15,226 INFO]  * tgt vocab size: 113.
[2021-03-25 09:44:15,228 INFO]  * src vocab size: 1082.
[2021-03-25 09:44:15,232 INFO] Building & saving validation data...
[2021-03-25 09:44:15,265 INFO] Building shard 0.
[2021-03-25 09:44:15,339 INFO]  * saving 0th valid data shard to data/preprocessed_data.valid.0.pt.


In [21]:
!python OpenNMT-py-legacy/train.py \
-encoder_type brnn \
-global_attention mlp \
-world_size 1 -gpu_ranks 0 \
-train_steps 30000 -save_checkpoint_steps 500 \
-valid_steps 500 \
-data data/preprocessed_data \
-save_model ./text2app_models/model
#-train_from /content/drive/MyDrive/Model_Testing/text2app_models/model_step_12500.pt \

[2021-03-25 09:44:16,483 INFO]  * src vocab size = 1082
[2021-03-25 09:44:16,483 INFO]  * tgt vocab size = 113
[2021-03-25 09:44:16,483 INFO] Building model...
[2021-03-25 09:44:19,295 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(1082, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 250, num_layers=2, dropout=0.3, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(113, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_context): Linear(in_features=500, out_features=500

#Test

In [22]:
best_model_dir = "./text2app_models/model_step_12500.pt"

In [23]:
!python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
data/src_test.txt -output data/pred.txt -replace_unk -verbose -beam_size 1

Streaming output truncated to the last 5000 lines.
SENT 4001: ['make', 'a', 'mobile', 'app', 'consisting', 'a', 'password', 'box', ',', 'and', 'a', 'music', 'with', 'source', 'string0', '.']
PRED 4001: <complist> <passwordtextbox> <player> string0 </player> </complist>
PRED SCORE: 0.0000

[2021-03-25 10:05:46,023 INFO] 
SENT 4002: ['create', 'mobile', 'app', 'having', 'a', 'ball', ',', 'a', 'pasword', 'text', 'box', ',', 'a', 'video', 'player', 'with', 'a', 'random', 'clip', ',', 'an', 'audio', 'player', 'with', 'source', 'string0', ',', 'a', 'textbox', ',', 'and', 'a', 'label', '.', 'if', 'the', 'circle', 'is', 'cornered,', 'play', 'the', 'music', 'player', '.']
PRED 4002: <complist> <ball> <passwordtextbox> <video_player> random_video_player_source </video_player> <player> string0 </player> <textbox> <label> label1 </label> </complist> <code> <ball1flung> <player1> <start> </player1> </ball1flung> </code>
PRED SCORE: 0.0000

[2021-03-25 10:05:46,023 INFO] 
SENT 4003: ['create', 'appl

In [24]:
f = open("data/pred.txt")
lines = f.readlines()
f.close()

In [26]:
pred = pd.DataFrame({'pred':lines})

In [27]:
len(pred)

5000

In [28]:
len(test)

5000

In [29]:
pred

,pred
0,<complist> <player> random_player_source </pla...
1,<complist> <ball> <textbox> <switch> switch1 <...
2,<complist> <timepicker> <player> string0 </pla...
3,<complist> <player> string0 </player> <video_p...
4,<complist> <textbox> <switch> switch1 </switch...
...,...
4995,<complist> <switch> string0 </switch> <text2sp...
4996,<complist> <timepicker> <player> string0 </pla...
4997,<complist> <passwordtextbox> <ball> <player> s...
4998,<complist> <accelerometer> <player> string0 </...


In [30]:
test[['SAR']]

,SAR
0,<complist> <player> random_player_source </pla...
1,<complist> <ball> <textbox> <switch> switch1 <...
2,<complist> <timepicker> <player> string0 </pla...
3,<complist> <player> string0 </player> <video_p...
4,<complist> <textbox> <switch> switch1 </switch...
...,...
4995,<complist> <switch> string0 </switch> <text2sp...
4996,<complist> <timepicker> <player> string0 </pla...
4997,<complist> <passwordtextbox> <ball> <player> s...
4998,<complist> <accelerometer> <player> string0 </...


In [31]:
def trim(s):
  return ' '.join(s.split())

In [32]:
import numpy as np

In [33]:
test_np = test['SAR'].apply(trim).to_numpy()
pred_np = pred['pred'].apply(trim).to_numpy()

In [34]:
correct = 1*(test_np == pred_np)

In [35]:
correct.sum()/len(correct)

0.8134

## Single data Test

In [ ]:
def is_Number(test):
  is_number = True
  try:
    float(test)
  except ValueError:
    is_number = False
  return is_number

In [ ]:
import nltk
def get_singular(word):
  p = nltk.PorterStemmer()
  a = p.stem(word)
  return a 

In [ ]:
def NL_mod(nl):
  vis_portion = nl.split(".")[0].split() 
  for i in range(len(vis_portion)):
    if is_Number(vis_portion[i]):
      if float(vis_portion[i]) > 1:
        component = get_singular(vis_portion[i+1])
        mod = ""
        for j in range(int(vis_portion[i])):
          mod += component
          mod += " "
        mod = mod.rstrip()
        rep = vis_portion[i] + " " + vis_portion[i + 1]
        nl = nl.replace(rep, mod, 1)
  return nl.lower()

In [ ]:
best_model_dir = '/content/drive/MyDrive/Model_Testing/text2app_models/model_step_3000.pt'

In [ ]:
def test(nl):
  nl = NL_mod(nl)
  with open("test.txt", "w") as f:
    f.write(nl)
  
  !python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
test.txt -output sar_to_compile.txt -replace_unk -verbose -beam_size 1
  out = open("sar_to_compile.txt", 'r')
  print(out)
  # return out

In [ ]:
nl = """Create an app with noise noise a text to speech , and button STRING0 . When the button is clicked, speak the text in the text box ."""
test(nl)

[2021-03-14 07:49:22,231 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
[2021-03-14 07:49:22,285 INFO] 
SENT 1: ['create', 'an', 'app', 'with', 'noise', 'noise', 'a', 'text', 'to', 'speech', ',', 'and', 'button', 'STRING0', '.', 'when', 'the', 'button', 'is', 'clicked,', 'speak', 'the', 'text', 'in', 'the', 'text', 'box', '.']
PRED 1: <start_vis> <text2speech> <button> STRING0 </button> <end_vis> <code> <button1clicked> <text2speech1> <textboxtext1> </text2speech1> </button1clicked> </code>
PRED SCORE: 0.0000

[2021-03-14 07:49:22,285 INFO] PRED AVG SCORE: 0.0000, PRED PPL: 1.0000
<_io.TextIOWrapper name='sar_to_compile.txt' mode='r' encoding='UTF-8'>


In [ ]:
from google.colab import files
files.download('sar_to_compile.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
best_model_dir = '/content/drive/MyDrive/Model_Testing/text2app_models/model_step_30000.pt'

In [ ]:
!python OpenNMT-py-legacy/translate.py -model $best_model_dir -src \
single_test.txt -output sar_to_compile.txt -replace_unk -verbose -beam_size 1

[2021-03-24 10:35:00,904 INFO] Translating shard 0.
[2021-03-24 10:35:00,963 INFO] 
SENT 1: ['create', 'an', 'app', 'with', 'a', 'button', ',', 'a', 'textbox', ',', 'and', 'a', 'text2speech', '.', 'when', 'the', 'button', 'is', 'pressed', ',', 'speak', 'the', 'textbox', 'text', '.']
PRED 1: <start_comp> <button> button1 </button> <textbox> <text2speech> <end_comp> <code> <button1clicked> <text2speech1> <textboxtext1> </text2speech1> </button1clicked> </code>
PRED SCORE: 0.0000

[2021-03-24 10:35:00,963 INFO] PRED AVG SCORE: 0.0000, PRED PPL: 1.0000


In [ ]:
import subprocess

def tranlsate(NL):
  nl_file = open("single_test.txt", "w")
  nl_file.write(NL)
  nl_file.close()
  s = "python OpenNMT-py-legacy/translate.py -model {} -src single_test.txt -output sar_to_compile.txt -replace_unk -verbose -beam_size 1".format(best_model_dir)
  subprocess.call(s.split())
  f = open("sar_to_compile.txt", 'r')
  sar = f.read().strip()
  f.close()
  subprocess.call('rm single_test.txt'.split())
  subprocess.call('rm sar_to_compile.txt'.split())
  return sar

In [ ]:
NL = "create an app with a button , a textbox , and a text2speech . when the button is pressed , speak the textbox text ."
tranlsate(NL)

'<start_comp> <button> button1 </button> <textbox> <text2speech> <end_comp> <code> <button1clicked> <text2speech1> <textboxtext1> </text2speech1> </button1clicked> </code>'

In [ ]:
!wget https://www.dropbox.com/s/du1w2kkbxkwv788/OpenNMT-py-legacy.zip
!unzip OpenNMT-py-legacy.zip

--2021-03-24 10:48:57--  https://www.dropbox.com/s/du1w2kkbxkwv788/OpenNMT-py-legacy.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/du1w2kkbxkwv788/OpenNMT-py-legacy.zip [following]
--2021-03-24 10:48:57--  https://www.dropbox.com/s/raw/du1w2kkbxkwv788/OpenNMT-py-legacy.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0e79fdb16f65e24fb4e89df7f5.dl.dropboxusercontent.com/cd/0/inline/BLQTO3Kqwucpta1Pqm4p0QRZiCn3jWfJwqkgQ7btUXdii7_tXgX_Fq-w7ASBEvLgOFN_0RF6wuh4tzhBflvgUpN25XiuDTS0ESKQLjPq0eYY0XJtts_TIgCM4ATZttqM4nRhtpGIdIr6esHFHD2AYRlI/file# [following]
--2021-03-24 10:48:58--  https://uc0e79fdb16f65e24fb4e89df7f5.dl.dropboxusercontent.com/cd/0/inline/BLQTO3Kqwucpta1Pqm4p0QRZiCn3jWfJwqkgQ7btUXdii7_tXgX_Fq-w7ASB